# Working with LLC90 data on SciServer (using OceanSpy)

<font size="3">This notebook is meant to demonstrate the workflow to use the data on the LLC90 grid (ECCOv4r4) on the Johns Hopkins University's [SciServer](https://www.sciserver.org/) Portal. It serves as a precurser to the LLC4320 tutorial by introducing basic Oceanspy and SciServer essentials to the user and explore the ECCOv4r4 dataset as a test case for data analysis. Here, we will simply try and show model-data comparison between some [WOCE](http://woceatlas.ucsd.edu/) sections in the Atlantic and the ECCOv4r4 data. 

&nbsp;&nbsp;&nbsp;&nbsp;<img src="img/oceanspy_logo_blue.png" style="width: 200px;"/>   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp; &nbsp;&nbsp; &nbsp; <img src="img/sciserver.png" style="width: 150px;"/> ; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<img src="img/poseidon_white.png" style="width: 150px;"/> &nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp;<img src="img/pangeo_logo.png" style="width: 250px;"/> &nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp;<img src="img/NSF_logo.png" style="width: 150px;"/> 

### Let's get started !

In [1]:
#Importing oceanspy and some other packages

import oceanspy as ospy
import xarray as xr
import matplotlib.pyplot as plt 
import cmocean
import cartopy.crs as ccrs
import numpy as np
import matplotlib.pylab as pylab

In [2]:
# Setting plotting parameters
params = {'figure.figsize':'20,8',
          'legend.fontsize': 'x-large',
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large',
          'text.usetex':True}
pylab.rcParams.update(params)

In [3]:
# This is to suppress warnings 
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Creating a dask client
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 100.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43082,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 100.00 GiB
Comm: tcp://127.0.0.1:36429,Total threads: 1
Dashboard: http://127.0.0.1:33468/status,Memory: 25.00 GiB
Nanny: tcp://127.0.0.1:45834,


# Loading the ECCOv4r4 dataset

<font size="3">This is where we initiate the oceanspy object and import the ECCOv4r4 data stored on SciServer. The SciServer catalog contains multiple publicly available datasets. A detailed description of the available datasets can be found here. https://oceanspy.readthedocs.io/en/latest/datasets.html. For more information on the ECCO dataset, please refer to ECCO Version 4 Release 4 Synopsis (https://doi.org/10.5281/zenodo.4533349)

In [5]:
%%time
od = ospy.open_oceandataset.from_catalog('ECCO')

Opening ECCO.
ECCO v4r4 3D dataset, ocean simulations on LLC90 grid
CPU times: user 2.65 s, sys: 759 ms, total: 3.41 s
Wall time: 9.03 s


In [6]:
# Let's see how big the dataset is !
print (str(od.dataset.nbytes * 1.0e-9) + ' GB')

145.518932476 GB


In [7]:
# some bookkeeping
_ds = od._ds.drop({'k', 'k_p1','k_l','k_u'})
od._ds = _ds

# LLC90 as OceanSpy object

<font size="3">This is what the oceanspy object looks like when we import the LLC_90 data in it's native grid, without any transformations into an interpolated lat-lon grid. 

In [8]:
od.dataset

<xarray.Dataset>
Dimensions:     (time: 312, Zl: 50, face: 13, Y: 90, X: 90, Z: 50, Xp1: 90,
                 Yp1: 90, time_midp: 311, Zp1: 51, Zu: 50, nv: 2)
Coordinates: (12/38)
    CS          (face, Y, X) float32 dask.array<chunksize=(13, 90, 90), meta=np.ndarray>
    Depth       (face, Y, X) float32 dask.array<chunksize=(13, 90, 90), meta=np.ndarray>
    PHrefC      (Z) float32 dask.array<chunksize=(50,), meta=np.ndarray>
    PHrefF      (Zp1) float32 dask.array<chunksize=(51,), meta=np.ndarray>
    SN          (face, Y, X) float32 dask.array<chunksize=(13, 90, 90), meta=np.ndarray>
    XC          (face, Y, X) float32 dask.array<chunksize=(13, 90, 90), meta=np.ndarray>
    ...          ...
    rAw         (face, Y, Xp1) float32 dask.array<chunksize=(13, 90, 90), meta=np.ndarray>
    rAz         (face, Yp1, Xp1) float32 dask.array<chunksize=(13, 90, 90), meta=np.ndarray>
  * time        (time) datetime64[ns] 1992-01-16T12:00:00 ... 2017-12-16
    time_bnds   (time, nv) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
  * time_midp   (time_midp) datetime64[ns] 1992-01-31T12:00:00 ... 2017-12-01
    timestep    (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
Dimensions without coordinates: nv
Data variables: (12/30)
    ADVr_SLT    (time, Zl, face, Y, X) float32 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    ADVr_TH     (time, Zl, face, Y, X) float32 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    ADVx_SLT    (time, Z, face, Y, Xp1) float32 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    ADVx_TH     (time, Z, face, Y, Xp1) float32 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    ADVy_SLT    (time, Z, face, Yp1, X) float32 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    ADVy_TH     (time, Z, face, Yp1, X) float32 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    ...          ...
    UVELMASS    (time, Z, face, Y, Xp1) float32 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    VVELMASS    (time, Z, face, Yp1, X) float32 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    WVELMASS    (time, Zl, face, Y, X) float32 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    oceFWflx    (time, face, Y, X) float32 dask.array<chunksize=(1, 13, 90, 90), meta=np.ndarray>
    oceQsw      (time, face, Y, X) float32 dask.array<chunksize=(1, 13, 90, 90), meta=np.ndarray>
    oceSPtnd    (time, Z, face, Y, X) float32 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
Attributes: (12/16)
    date_created:               Mon Dec 30 11:13:26 2019
    geospatial_lat_max:         90.0
    geospatial_lat_min:         -90.0
    geospatial_lon_max:         180.0
    geospatial_lon_min:         -179.9991912841797
    geospatial_vertical_max:    -5.0
    ...                         ...
    title:                      ECCOv4 MITgcm grid information
    OceanSpy_parameters:        {'rSphere': 6371.0, 'eq_state': 'jmd95', 'rho...
    OceanSpy_name:              ECCO_v4r4
    OceanSpy_description:       ECCO v4r4 3D dataset, ocean simulations on LL...
    OceanSpy_grid_coords:       {'Y': {'Y': None, 'Yp1': 0.5}, 'X': {'X': Non...
    OceanSpy_face_connections:  {'face': {0: {'X': ((12, 'Y', False), (3, 'X'...

# Using the `ospy.cutout` function to transform the data. 

<font size="3">Since the dataset is in its native LLC grid distributed across 13 tiles/faces, we need to transform the data into a regular lat-lon grid to be able to use it with OceanSpy. For now, the `ospy.cutout` function helps us achieve that. Note that the ECCO data is also available as interpolated monthly fields on regular lat-lon grids, however, we take the example of the LLC_90 to demonstrate this transformation because as we move towards higher resolution datasets, such as the LLC2160 and LLC4320, we will have to transform it from the native grid, since those are not available as interpolated datasets. 

In [ ]:
%%time
## Parameters


varList = ['THETA', 'SALT', 'Depth']
YRange = [20,80] # degrees latitude
XRange = [-90, 20] # longitude
add_Hbdr = False
mask_outside = True
ZRange = None
add_Vbdr = False
timeRange = ['1992', '2017']
timeFreq= None
sampMethod = 'snapshot'
dropAxes = False

# Transformation parameters
centered= 'Atlantic'
transformation = 'arctic_crown'

## Carrying out the cutout
cut_od = ospy.subsample.cutout(od,varList,YRange,XRange,add_Hbdr,mask_outside,
                               ZRange,add_Vbdr,timeRange,timeFreq,sampMethod,dropAxes, 
                               centered=centered, transformation=transformation)

In [ ]:
# Let's see how big the cutout is
print (str(cut_od.dataset.nbytes * 1.0e-9) + ' GB')

In [ ]:
# How does is look ?
cut_od._ds

# Observational data (WOCE sections)

<font size="3">As a test case, we will simply take the A01 section in the North Atlantic and plot the salinity and potential temperature vertical sections. We will then use this to validate the model data. This section needs expansion with information about the campaigns and how to fetch data.<br>

![WOCE Atlantic section](img/woce_stations_final.jpg "WOCE image")

In [ ]:
# opening files from CCHDO https://cchdo.ucsd.edu/
ds_a1 = xr.open_dataset('test_data/06MT30_3_ctd.nc')
ds_a5 = xr.open_dataset('test_data/29HE06_1_ctd.nc')

# Creating the A01 section for ECCO using OceanSpy

In [ ]:
# Masking the depth
cut_od._ds['Depth_masked'] = cut_od._ds['Depth']
depth_masked = cut_od._ds['Depth_masked']
depth_masked = depth_masked.where(depth_masked>0)
cut_od._ds['Depth_masked'] = depth_masked

# Making a survey using OceanSpy !

<font size="3">We simply provide the lat-lon information to the `ospy.subsample.survey_stations` function along with any other relavant information required, such as depth levels and the variables that need to be included in the analysis. 

In [ ]:
time_range_A01 = ['1994-11', '1994-12']
time_range_A05 = ['1992-07', '1992-08']

In [ ]:
%%time
## ecco A01 and A05 information 

lat_a1 = ds_a1.latitude
lon_a1 = ds_a1.longitude
depth_a1 = [-ds_a1.btm_depth.min(), -ds_a1.btm_depth.max()]


lat_a5 = ds_a5.latitude
lon_a5 = ds_a5.longitude
depth_a5 = [ds_a5.btm_depth.min(), -ds_a5.btm_depth.max()]

# Extract ship survey and fields used by this notebook
od_surv_a1 = cut_od.subsample.survey_stations(Xsurv=lon_a1,
                                              Ysurv=lat_a1,
                                              ZRange=depth_a1,
                                              timeRange = time_range_A01,
                                              varList=["THETA", 'SALT'])

od_surv_a5 = cut_od.subsample.survey_stations(Xsurv=lon_a5,
                                              Ysurv=lat_a5,
                                              ZRange=depth_a5,
                                              timeRange = time_range_A05,
                                              varList=["THETA", 'SALT'])

# Visualizing the sections

<font size="3">We can now visualize the section A01 and compare the observations with the model data !

In [ ]:
# Here we can set projections for the visuals
cut_od = cut_od.set_projection('Orthographic', central_longitude=-40)

In [ ]:
%%time
# Plot map of survey locations

fig= plt.figure(frameon=True)
ax = cut_od.plot.horizontal_section(varName="Depth_masked", 
                                    plotType='contourf',
                                    levels=np.linspace(0,6000,50),
                                    cmap = cmocean.cm.deep,
                                    use_coords=True)

XC_a1 = od_surv_a1.dataset["XC"].squeeze()
YC_a1 = od_surv_a1.dataset["YC"].squeeze()

XC_a5 = od_surv_a5.dataset["XC"].squeeze()
YC_a5 = od_surv_a5.dataset["YC"].squeeze()

ax.set_extent([-85, 15, 20, 80], crs=ccrs.PlateCarree())

a1_line = ax.plot(XC_a1, YC_a1, 's', markersize=5, markerfacecolor='r',
             markeredgewidth=0.5, markeredgecolor='w', transform=ccrs.PlateCarree())
a5_line = ax.plot(XC_a5, YC_a5, 's', markersize=5, markerfacecolor='r',
             markeredgewidth=0.5, markeredgecolor='w', transform=ccrs.PlateCarree())

plt.figtext(0.44,0.60, 'A01', fontsize=24, color='white')
plt.figtext(0.44,0.29, 'A05', fontsize=24, color='white')

ax.patch.set_facecolor('black')

# Visualizing the vertical sections

<font size="3">A01 section in ECCO and observations 

In [ ]:
od_surv_a1._ds = od_surv_a1._ds.where(od_surv_a1._ds.SALT>34)
od_surv_a5._ds = od_surv_a5._ds.where(od_surv_a5._ds.SALT>34)

## Salinity

In [ ]:
%%time
import numpy as np
from cartopy.crs import PlateCarree
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# To toggle between the A1 and A5 sections, simply uncomment the commented lines and make sure you comment out the lines for the current case.

# Create subplots
fig = plt.figure()
ax1 = plt.subplot2grid((2,1), (0,0), rowspan=1, colspan=1)
ax2 = plt.subplot2grid((2,1), (1,0), rowspan=1, colspan=1)

levels_salt_a5 = np.linspace(34,ds_a5.ctd_salinity.max(),50)
levels_salt_a1 = np.linspace(34,ds_a1.ctd_salinity.max(),50)

c0=ax1.contourf(ds_a5.ctd_salinity.transpose(), levels_salt_a5, cmap=cmocean.cm.haline, extend='both' )
#c0=ax1.contourf(ds_a1.ctd_salinity.transpose(), levels_salt_a1, cmap=cmocean.cm.haline, extend='both' )

ax1.invert_yaxis()

ax1.set_xticks([])
ax1.set_yticklabels(['0','-1000','-2000', '-3000', '-4000', '-5000', '-6000'])
#ax1.set_yticklabels(['0','-1000','-2000', '-3000', '-4000'])
ax1.set_facecolor('black')
#ax1.set_title('A01 in observations (winter 1994)')
ax1.set_title('A05 in observations (winter 1992)')


ax2 = od_surv_a5.plot.vertical_section(
    varName="SALT",
    robust=True,
    subsamp_kwargs={'time':time_range_A05[0]},
    levels=levels_salt_a5,
    meanAxes='time',
    cmap=cmocean.cm.haline,
    add_colorbar=False,
    ax=ax2)
    
ax2.patch.set_facecolor('black')
#ax2.set_title('A01 in ECCOv4r4 (winter 1994)')
ax2.set_title('A05 in ECCOv4r4 (winter 1992)')

plt.figtext(0.84,0.83, '$psu$', fontsize=24)

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.82, 0.15, 0.01, 0.7])
fig.colorbar(c0, cax=cbar_ax)

## Temperature

In [ ]:
%%time
# To toggle between the A1 and A5 sections, simply uncomment the commented lines and make sure you comment out the lines for the current case.

fig = plt.figure()
ax1 = plt.subplot2grid((2,1), (0,0), rowspan=1, colspan=1)
ax2 = plt.subplot2grid((2,1), (1,0), rowspan=1, colspan=1)

levels_temp_a5 = np.linspace(0,ds_a5.ctd_temperature.max(),50)
#levels_temp_a1 = np.linspace(0,ds_a5.ctd_temperature.max(),50)

c0=ax1.contourf(ds_a5.ctd_temperature.transpose(), levels_temp_a5, cmap=cmocean.cm.thermal, extend='both' )
#c0=ax1.contourf(ds_a1.ctd_temperature_unk.transpose(), levels_temp_a1, cmap=cmocean.cm.thermal, extend='both' )

ax1.invert_yaxis()

ax1.set_xticks([])
ax1.set_yticklabels(['0','-1000','-2000', '-3000', '-4000', '-5000', '-6000'])
#ax1.set_yticklabels(['0','-1000','-2000', '-3000', '-4000'])
ax1.set_facecolor('black')
#ax1.set_title('A01 in observations (winter 1994)')
ax1.set_title('A05 in observations (winter 1992)')


# Change the dataset when toggling between A1 and A5
ax2 = od_surv_a5.plot.vertical_section(
    varName="THETA",
    robust=True,
    subsamp_kwargs={'time':time_range_A05[0]},
    levels=levels_temp_a5,
    meanAxes='time',
    cmap=cmocean.cm.thermal,
    add_colorbar=False,
    ax=ax2)
    
ax2.patch.set_facecolor('black')
#ax2.set_title('A01 in ECCOv4r4 (winter 1994)')
ax2.set_title('A05 in ECCOv4r4 (winter 1994)')


plt.figtext(0.84,0.83, '$^{\circ}C$', fontsize=18)

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.82, 0.15, 0.01, 0.7])
fig.colorbar(c0, cax=cbar_ax)